In [ ]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path

# Allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
%matplotlib inline

# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in the CSV files
googl_df = pd.read_csv("Resources/googl_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
nvda_df = pd.read_csv("Resources/nvda_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
mmm_df = pd.read_csv("Resources/mmm_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
pg_df = pd.read_csv("Resources/pg_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)

In [3]:
# Drop unnecessary columns
column_drops =['actual_returns', 'sma_fast', 'sma_fast30', 'sma_slow', 'ema_fast', 'ema_fast30', 'ema_slow']

for df in (googl_df, nvda_df, mmm_df, pg_df):
    df.drop(columns=column_drops, inplace=True)

In [4]:
# Create a list of individual stock DataFrame names and columns to feed loops
ticker_data = [(googl_df, "GOOGL Adj. Close"),(nvda_df, "NVDA Adj. Close"),(mmm_df, "MMM Adj. Close"),(pg_df, "PG Adj. Close")]

In [5]:
# Loop to add actual_returns based on percentage change of Adj. Close column
for df_name, column_name in ticker_data:
    df_name["actual_returns"] = df_name[column_name].pct_change()

In [6]:
# Create a list of tuples to feed the loop for the individual stock DataFrames and financial ratios
ticker_data_all = [
    (googl_df, [("GOOGL P/S (LTM)", "ps"), ("GOOGL P/FCF (LTM)", "pfcf"), ("GOOGL P/E (LTM)", "pe")]),
    (nvda_df, [("NVDA P/S (LTM)", "ps"), ("NVDA P/FCF (LTM)", "pfcf"), ("NVDA P/E (LTM)", "pe")]),
    (mmm_df, [("MMM P/S (LTM)", "ps"), ("MMM P/FCF (LTM)", "pfcf"), ("MMM P/E (LTM)", "pe")]),
    (pg_df, [("PG P/S (LTM)", "ps"), ("PG P/FCF (LTM)", "pfcf"), ("PG P/E (LTM)", "pe")]),
]

In [7]:
# Loop to generate SMA and EMA (10-, 30-, 100-day windows) for each ticker (all ratios)
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        # Generate SMA and EMA (10-, 30-, 100-day windows) for each ticker (all ratios)
        df[f"{prefix}_sma_fast"] = df[column_name].rolling(window=10).mean()
        df[f"{prefix}_sma_fast30"] = df[column_name].rolling(window=30).mean()
        df[f"{prefix}_sma_slow"] = df[column_name].rolling(window=100).mean()
        df[f"{prefix}_ema_fast"] = df[column_name].ewm(span=10, adjust=False).mean()
        df[f"{prefix}_ema_fast30"] = df[column_name].ewm(span=30, adjust=False).mean()
        df[f"{prefix}_ema_slow"] = df[column_name].ewm(span=100, adjust=False).mean()

#### Storing individual stock DataFrames with SMA and EMA (10-, 30-, 100-day windows) to Resources folder

In [8]:
# Write individual stock DFs w/ SMA, EMA, and Bollinger Bands/RSI generated with all ratios data to CSV files
googl_df.to_csv('Resources/googl_ratio_indicators.csv', index=True)
nvda_df.to_csv('Resources/nvda_ratio_indicators.csv', index=True)
mmm_df.to_csv('Resources/mmm_ratio_indicators.csv', index=True)
pg_df.to_csv('Resources/pg_ratio_indicators.csv', index=True)